In [157]:
import folium
import pandas as pd
import numpy as np
from haversine import haversine
from tqdm import tqdm

In [47]:
market_region = pd.read_csv('./2022상권데이터 최종.csv', encoding='EUC-KR')
market_region

,연도_분기,상권_코드_명,상권_코드,점포_수,long,lat,분기당_매출_금액,분기당_매출_건수,주말_매출_금액,주말_매출_건수,...,시간대_건수~06_매출_건수,시간대_건수~11_매출_건수,시간대_건수~14_매출_건수,시간대_건수~17_매출_건수,시간대_건수~21_매출_건수,시간대_건수~24_매출_건수,개업_점포_수,폐업_점포_수,유사_업종_점포_수,프랜차이즈_점포_수
0,2022_2분기,이태원 관광특구,1001491,1702,126.994377,37.534435,125128808539,4345470,47086678652,1967156,...,830085,285058,610322,702249,1128223,789533,57,29,1784,82
1,2022_2분기,명동 남대문 북창동 다동 무교동 관광특구,1001492,8228,126.981856,37.564149,414295939101,15035663,79821252531,2646506,...,143642,2662173,5494707,3382050,2708788,644303,110,80,8819,591
2,2022_2분기,동대문패션타운 관광특구,1001493,11708,127.011025,37.567312,226971025345,7821510,68228002280,2010722,...,471997,757424,1779159,1769777,2317560,725593,54,106,11900,192
3,2022_2분기,종로?청계 관광특구,1001494,5045,126.997374,37.569998,270349786683,7484371,61175003235,2134062,...,158227,777701,2411263,2013432,1606240,517508,73,52,5270,225
4,2022_2분기,잠실 관광특구,1001495,2232,127.115275,37.516476,109710843704,5520817,35757879427,1942939,...,289797,565226,1154183,1115855,1611464,784292,50,35,2463,231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1644,2022_2분기,고분다리전통시장,2130322,135,127.133023,37.542208,3958481135,215630,1189584202,62873,...,7778,23779,39508,46713,76853,20999,1,1,140,5
1645,2022_2분기,둔촌역전통시장,2130323,200,127.134310,37.527818,13428152555,598080,4111707031,191826,...,11088,49401,113462,152228,230364,41537,4,3,212,12
1646,2022_2분기,길동복조리시장,2130324,133,127.143563,37.538925,5143279432,263420,1719330380,84663,...,447,24169,62634,75555,93222,7393,2,3,139,6
1647,2022_2분기,명일전통시장,2130325,98,127.145337,37.549293,4766099294,259944,1380847128,78429,...,52,25060,59796,75815,92857,6364,6,3,101,3


# http://www.hotelrestaurant.co.kr/news/article.html?no=1021
# 상권의 위치별 1차상권은 500m범위로 잡았습니다.

In [48]:
my_map = folium.Map(location=[37.5502, 126.982], zoom_start=11, title="Stamen Toner")

for each in range(len(market_region)):
    marker_circle = folium.Circle([market_region['lat'][each], market_region['long'][each]],
                                       radius = 500,
                                        color = 'skyblue',
                                        popup = market_region['상권_코드_명'][each],
                                        fill_color = 'skyblue'
                                       )
    marker_circle.add_to(my_map)

In [49]:
my_map

# 분기당 매출건수 상위 100위인 곳만 뽑았다.

In [64]:
market_region_top = market_region.sort_values(by='분기당_매출_건수', ascending=False)
market_region_top = market_region_top.reset_index()
market_region_top = market_region_top.head(100)

In [65]:
my_map = folium.Map(location=[37.5502, 126.982], zoom_start=11, title="Stamen Toner")

for each in range(len(market_region_top)):
    marker_circle = folium.Circle([market_region_top['lat'][each], market_region_top['long'][each]],
                                       radius = 500,
                                        color = 'skyblue',
                                        popup = market_region['상권_코드_명'][each],
                                        fill_color = 'skyblue'
                                       )
    marker_circle.add_to(my_map)

In [66]:
my_map

# 상권이 이렇게나 많습니다.
# 분기별매출, 분기별건수가 얼마 이상인 곳만 보는 것도 방법이고
# 지하철역 주변에 상권이 얼마나 있는가로 들어가도 좋을거 같습니다

# 명분이 없다 왜 상위 100개인건가 
https://m.blog.naver.com/budam457/222030544465


대형상권으로 보자 일매출이 10억 이상이 상권

In [73]:
market_region_everyday = market_region
market_region_everyday['일매출금액'] = market_region['분기당_매출_금액']/91
market_region_everyday

,연도_분기,상권_코드_명,상권_코드,점포_수,long,lat,분기당_매출_금액,분기당_매출_건수,주말_매출_금액,주말_매출_건수,...,시간대_건수~11_매출_건수,시간대_건수~14_매출_건수,시간대_건수~17_매출_건수,시간대_건수~21_매출_건수,시간대_건수~24_매출_건수,개업_점포_수,폐업_점포_수,유사_업종_점포_수,프랜차이즈_점포_수,일매출금액
0,2022_2분기,이태원 관광특구,1001491,1702,126.994377,37.534435,125128808539,4345470,47086678652,1967156,...,285058,610322,702249,1128223,789533,57,29,1784,82,1.375042e+09
1,2022_2분기,명동 남대문 북창동 다동 무교동 관광특구,1001492,8228,126.981856,37.564149,414295939101,15035663,79821252531,2646506,...,2662173,5494707,3382050,2708788,644303,110,80,8819,591,4.552703e+09
2,2022_2분기,동대문패션타운 관광특구,1001493,11708,127.011025,37.567312,226971025345,7821510,68228002280,2010722,...,757424,1779159,1769777,2317560,725593,54,106,11900,192,2.494187e+09
3,2022_2분기,종로?청계 관광특구,1001494,5045,126.997374,37.569998,270349786683,7484371,61175003235,2134062,...,777701,2411263,2013432,1606240,517508,73,52,5270,225,2.970877e+09
4,2022_2분기,잠실 관광특구,1001495,2232,127.115275,37.516476,109710843704,5520817,35757879427,1942939,...,565226,1154183,1115855,1611464,784292,50,35,2463,231,1.205614e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1644,2022_2분기,고분다리전통시장,2130322,135,127.133023,37.542208,3958481135,215630,1189584202,62873,...,23779,39508,46713,76853,20999,1,1,140,5,4.349979e+07
1645,2022_2분기,둔촌역전통시장,2130323,200,127.134310,37.527818,13428152555,598080,4111707031,191826,...,49401,113462,152228,230364,41537,4,3,212,12,1.475621e+08
1646,2022_2분기,길동복조리시장,2130324,133,127.143563,37.538925,5143279432,263420,1719330380,84663,...,24169,62634,75555,93222,7393,2,3,139,6,5.651955e+07
1647,2022_2분기,명일전통시장,2130325,98,127.145337,37.549293,4766099294,259944,1380847128,78429,...,25060,59796,75815,92857,6364,6,3,101,3,5.237472e+07


In [80]:
big_market_region = market_region_everyday.loc[market_region_everyday['일매출금액'] >= 1000000000]
big_market_region

,연도_분기,상권_코드_명,상권_코드,점포_수,long,lat,분기당_매출_금액,분기당_매출_건수,주말_매출_금액,주말_매출_건수,...,시간대_건수~11_매출_건수,시간대_건수~14_매출_건수,시간대_건수~17_매출_건수,시간대_건수~21_매출_건수,시간대_건수~24_매출_건수,개업_점포_수,폐업_점포_수,유사_업종_점포_수,프랜차이즈_점포_수,일매출금액
0,2022_2분기,이태원 관광특구,1001491,1702,126.994377,37.534435,125128808539,4345470,47086678652,1967156,...,285058,610322,702249,1128223,789533,57,29,1784,82,1.375042e+09
1,2022_2분기,명동 남대문 북창동 다동 무교동 관광특구,1001492,8228,126.981856,37.564149,414295939101,15035663,79821252531,2646506,...,2662173,5494707,3382050,2708788,644303,110,80,8819,591,4.552703e+09
2,2022_2분기,동대문패션타운 관광특구,1001493,11708,127.011025,37.567312,226971025345,7821510,68228002280,2010722,...,757424,1779159,1769777,2317560,725593,54,106,11900,192,2.494187e+09
3,2022_2분기,종로?청계 관광특구,1001494,5045,126.997374,37.569998,270349786683,7484371,61175003235,2134062,...,777701,2411263,2013432,1606240,517508,73,52,5270,225,2.970877e+09
4,2022_2분기,잠실 관광특구,1001495,2232,127.115275,37.516476,109710843704,5520817,35757879427,1942939,...,565226,1154183,1115855,1611464,784292,50,35,2463,231,1.205614e+09
5,2022_2분기,강남 마이스 관광특구,1001496,823,127.060059,37.511003,179896455786,4350688,63689858507,1286446,...,348565,1266471,1128653,1420871,180171,4,9,918,95,1.976884e+09
1100,2022_2분기,종각역,2120006,2108,126.983496,37.570339,129766749443,4925993,19940977032,813067,...,693229,1658471,856633,1179512,436700,41,29,2323,215,1.426008e+09
1103,2022_2분기,종로3가역,2120009,3523,126.990921,37.572763,310451448682,4570486,66420692723,1216571,...,514208,1187813,943609,1208236,502558,59,37,3649,126,3.411554e+09
1122,2022_2분기,명동(명동거리),2120028,2057,126.984256,37.563230,116841328584,5096760,28821771828,1164156,...,778751,1713059,1089612,1171949,287418,24,29,2324,267,1.283971e+09
1125,2022_2분기,을지로3가역,2120031,1169,126.991522,37.566429,115496672040,6381448,16256380307,1300636,...,1319186,2313480,1327134,978041,337423,19,20,1228,59,1.269194e+09


# 55개는 조금 적은감이 있다.
- 상권의 규모는 결국 총 유동인구의 수가 중요하다고 한다. 그래 그냥 유동인구로 처리해보자

# 대형상권은 : 하루 유동인구가 10만명이상
# 중형상권은 : 하루 유동인구가 2만명이상

- 우선 버스, 지하철, 주차장 데이터를 들고오자

- 모든 상권 주변에 유동인구를 구해보자 상권의 범위는 500m로 잡았다

In [90]:
car = pd.read_csv('./park_state.csv')
car_data = car[['위도', '경도', '총 주차면']]
car_data

,위도,경도,총 주차면
0,37.649279,127.040474,71
1,37.643917,127.058567,177
2,37.569939,127.035145,54
3,37.515205,126.915116,98
4,37.525526,126.895794,190
...,...,...,...
1088,37.622745,127.018428,95
1089,37.633227,127.020711,80
1090,37.612180,126.956535,40
1091,37.507334,126.891653,65


In [98]:
bus = pd.read_csv('./2020_10_bus_data_final.csv')
bus_data = bus[['X좌표', 'Y좌표', '총 승차 인원', '총 하차 인원', '환승 가능한 버스의 수']]
bus_data

,X좌표,Y좌표,총 승차 인원,총 하차 인원,환승 가능한 버스의 수
0,126.987750,37.569765,12190,13093,4
1,126.996566,37.579183,61055,54669,17
2,126.998340,37.582671,93098,70965,17
3,126.987613,37.568579,18248,17526,8
4,127.001744,37.586243,41183,68901,16
...,...,...,...,...,...
10493,127.151715,37.558546,728,4867,1
10494,127.153072,37.555506,1569,5534,1
10495,127.153378,37.555506,6383,2968,1
10496,127.152546,37.552350,5022,1706,1


In [119]:
# 환승가능한 지하철역은 어떻게 처리할거야?
subway = pd.read_csv('./metro_2022_10_location.csv', encoding = "EUC-KR")
subway


,Unnamed: 0,사용월,호선명,지하철역,출근시간(04~09) 승차,출근시간(07~10) 하차,퇴근시간(16~20) 승차,퇴근시간(17~21) 하차,점심시간,저녁시간,늦은시간퇴근(20~02) 승차,총 승차 인원,총 하차 인원,월,연도,lat,lng
0,0,202210,9호선,흑석,54931,53301,81702,81208,53412,35553,37242,273661,278426,10,2022,37.508770,126.963708
1,1,202210,경의선,효창공원앞,19655,21537,33229,24689,21944,17407,13602,90735,99249,10,2022,37.539261,126.961351
2,2,202210,6호선,효창공원앞,61192,39437,64600,58864,35996,30046,28665,221619,206279,10,2022,37.539261,126.961351
3,3,202210,4호선,회현,26095,247505,365489,61824,305703,157103,18577,746376,771983,10,2022,37.558514,126.978246
4,4,202210,경원선,회룡,155874,37024,52100,152665,20594,57891,66456,383494,379285,10,2022,37.724846,127.046895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,445,202210,7호선,가산디지털단지,75229,392535,659751,106468,411921,224992,41783,1100489,1090286,10,2022,37.481072,126.882343
446,446,202210,경부선,가산디지털단지,54534,159397,242111,91578,168761,109569,41888,456467,542963,10,2022,37.481072,126.882343
447,447,202210,8호선,가락시장,37426,48359,71155,61923,49401,40174,24963,218030,242681,10,2022,37.492522,127.118234
448,448,202210,3호선,가락시장,61056,44240,76234,73089,40509,36952,31820,256152,253412,10,2022,37.492522,127.118234


In [120]:
subway_ho = subway['지하철역'].value_counts()
subway_ho = pd.DataFrame(subway_ho)

In [121]:
subway_ho = subway_ho.reset_index()
subway_ho.rename(columns={subway_ho.columns[1]: "환승가능호선"}, inplace=True)
subway_ho.rename(columns={subway_ho.columns[0]: "지하철역"}, inplace=True)
subway_ho

,지하철역,환승가능호선
0,서울역,5
1,공덕,4
2,동대문역사문화공원,3
3,종로3가,3
4,디지털미디어시티,3
...,...,...
360,신정네거리,1
361,신중동,1
362,신풍,1
363,신흥,1


In [124]:
subway_pivot = subway.pivot_table(subway, index=["지하철역"], aggfunc = [np.sum] )
subway_pivot.columns = ['/'.join(col) for col in subway_pivot.columns]
subway_pivot = subway_pivot.reset_index()

# 위도 경도 더해버렸자나

subway_pro =pd.merge(subway_pivot , subway_ho, left_on='지하철역', right_on='지하철역', how='inner')
subway_pro = subway_pro[['지하철역', '환승가능호선', 'sum/총 승차 인원', 'sum/총 하차 인원']]
subway_pro

/tmp/ipykernel_6679/2963440509.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  subway_pivot = subway.pivot_table(subway, index=["지하철역"], aggfunc = [np.sum] )


,지하철역,환승가능호선,sum/총 승차 인원,sum/총 하차 인원
0,4.19민주묘지,1,114239,108826
1,가락시장,2,474182,496093
2,가산디지털단지,2,1556956,1633249
3,가양,1,619602,603098
4,가오리,1,125605,120499
...,...,...,...,...
360,회기,1,828890,799460
361,회룡,1,383494,379285
362,회현,1,746376,771983
363,효창공원앞,2,312354,305528


In [129]:
subway_location = subway[['지하철역', 'lat', 'lng']]
subway_location_2 = subway_location.drop_duplicates()
subway_location_2

,지하철역,lat,lng
0,흑석,37.508770,126.963708
1,효창공원앞,37.539261,126.961351
3,회현,37.558514,126.978246
4,회룡,37.724846,127.046895
5,회기,37.589460,127.057583
...,...,...,...
443,가오리,37.641701,127.016792
444,가양,37.561391,126.854456
445,가산디지털단지,37.481072,126.882343
447,가락시장,37.492522,127.118234


In [130]:
subway_data = pd.merge(subway_pro , subway_location_2, left_on='지하철역', right_on='지하철역', how='inner')
subway_data

,지하철역,환승가능호선,sum/총 승차 인원,sum/총 하차 인원,lat,lng
0,4.19민주묘지,1,114239,108826,37.649500,127.013700
1,가락시장,2,474182,496093,37.492522,127.118234
2,가산디지털단지,2,1556956,1633249,37.481072,126.882343
3,가양,1,619602,603098,37.561391,126.854456
4,가오리,1,125605,120499,37.641701,127.016792
...,...,...,...,...,...,...
360,회기,1,828890,799460,37.589460,127.057583
361,회룡,1,383494,379285,37.724846,127.046895
362,회현,1,746376,771983,37.558514,126.978246
363,효창공원앞,2,312354,305528,37.539261,126.961351


In [131]:
subway_data
bus_data
car_data

,위도,경도,총 주차면
0,37.649279,127.040474,71
1,37.643917,127.058567,177
2,37.569939,127.035145,54
3,37.515205,126.915116,98
4,37.525526,126.895794,190
...,...,...,...
1088,37.622745,127.018428,95
1089,37.633227,127.020711,80
1090,37.612180,126.956535,40
1091,37.507334,126.891653,65


# 상권내 모든 주차장에 사람이 1.5명씩 타고 있다 가정

In [158]:
market_region_human = market_region[['상권_코드_명', '점포_수', '분기당_매출_금액', '분기당_매출_건수', 'long', 'lat']]
market_region_human['유동인구'] = 0
market_region_human['환승가능한버스노선'] = 0
market_region_human['환승가능지하철'] =0
market_region_human

/tmp/ipykernel_6679/3415665858.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market_region_human['유동인구'] = 0
/tmp/ipykernel_6679/3415665858.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market_region_human['환승가능한버스노선'] = 0
/tmp/ipykernel_6679/3415665858.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

,상권_코드_명,점포_수,분기당_매출_금액,분기당_매출_건수,long,lat,유동인구,환승가능한버스노선,환승가능지하철
0,이태원 관광특구,1702,125128808539,4345470,126.994377,37.534435,0,0,0
1,명동 남대문 북창동 다동 무교동 관광특구,8228,414295939101,15035663,126.981856,37.564149,0,0,0
2,동대문패션타운 관광특구,11708,226971025345,7821510,127.011025,37.567312,0,0,0
3,종로?청계 관광특구,5045,270349786683,7484371,126.997374,37.569998,0,0,0
4,잠실 관광특구,2232,109710843704,5520817,127.115275,37.516476,0,0,0
...,...,...,...,...,...,...,...,...,...
1644,고분다리전통시장,135,3958481135,215630,127.133023,37.542208,0,0,0
1645,둔촌역전통시장,200,13428152555,598080,127.134310,37.527818,0,0,0
1646,길동복조리시장,133,5143279432,263420,127.143563,37.538925,0,0,0
1647,명일전통시장,98,4766099294,259944,127.145337,37.549293,0,0,0


In [159]:
market_region_human

,상권_코드_명,점포_수,분기당_매출_금액,분기당_매출_건수,long,lat,유동인구,환승가능한버스노선,환승가능지하철
0,이태원 관광특구,1702,125128808539,4345470,126.994377,37.534435,0,0,0
1,명동 남대문 북창동 다동 무교동 관광특구,8228,414295939101,15035663,126.981856,37.564149,0,0,0
2,동대문패션타운 관광특구,11708,226971025345,7821510,127.011025,37.567312,0,0,0
3,종로?청계 관광특구,5045,270349786683,7484371,126.997374,37.569998,0,0,0
4,잠실 관광특구,2232,109710843704,5520817,127.115275,37.516476,0,0,0
...,...,...,...,...,...,...,...,...,...
1644,고분다리전통시장,135,3958481135,215630,127.133023,37.542208,0,0,0
1645,둔촌역전통시장,200,13428152555,598080,127.134310,37.527818,0,0,0
1646,길동복조리시장,133,5143279432,263420,127.143563,37.538925,0,0,0
1647,명일전통시장,98,4766099294,259944,127.145337,37.549293,0,0,0


In [160]:
#37먼저
for i in tqdm(range(len(market_region_human))):
    market_region_location =  (market_region_human['lat'][i], market_region_human['long'][i])
    
    for each in range(len(bus_data)):
        bus_data_location = (bus_data['Y좌표'][each], bus_data['X좌표'][each])
        a = haversine(market_region_location, bus_data_location, unit = 'm')
        if a < 500:
            market_region_human['유동인구'][i] = market_region_human['유동인구'][i] +  bus_data['총 하차 인원'][each]
            market_region_human['환승가능한버스노선'][i] = market_region_human['환승가능한버스노선'][i] +  bus_data['환승 가능한 버스의 수'][each]
    
    for each in range(len(subway_data)):
        subway_data_location = (subway_data['lat'][each], subway_data['lng'][each])
        a = haversine(market_region_location, subway_data_location, unit = 'm')
        if a < 500:
            market_region_human['유동인구'][i] = market_region_human['유동인구'][i] +  subway_data['sum/총 하차 인원'][each]
            market_region_human['환승가능지하철'][i] = market_region_human['환승가능지하철'][i] +  subway_data['환승가능호선'][each]
    
    for each in range(len(car_data)):
        car_data_location = (car_data['위도'][each], car_data['경도'][each])
        a = haversine(market_region_location, car_data_location, unit = 'm')
        if a < 500:
            market_region_human['유동인구'][i] = market_region_human['유동인구'][i] +  (car_data['총 주차면'][each] * 1.5)

  0%|                                                  | 0/1649 [00:00<?, ?it/s]/tmp/ipykernel_6679/1117525198.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market_region_human['유동인구'][i] = market_region_human['유동인구'][i] +  bus_data['총 하차 인원'][each]
/tmp/ipykernel_6679/1117525198.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market_region_human['환승가능한버스노선'][i] = market_region_human['환승가능한버스노선'][i] +  bus_data['환승 가능한 버스의 수'][each]
/tmp/ipykernel_6679/1117525198.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pa

In [161]:
market_region_human

,상권_코드_명,점포_수,분기당_매출_금액,분기당_매출_건수,long,lat,유동인구,환승가능한버스노선,환승가능지하철
0,이태원 관광특구,1702,125128808539,4345470,126.994377,37.534435,756654.5,42,1
1,명동 남대문 북창동 다동 무교동 관광특구,8228,414295939101,15035663,126.981856,37.564149,3381223.0,212,3
2,동대문패션타운 관광특구,11708,226971025345,7821510,127.011025,37.567312,2278555.0,114,5
3,종로?청계 관광특구,5045,270349786683,7484371,126.997374,37.569998,1695851.5,139,3
4,잠실 관광특구,2232,109710843704,5520817,127.115275,37.516476,377853.0,32,2
...,...,...,...,...,...,...,...,...,...
1644,고분다리전통시장,135,3958481135,215630,127.133023,37.542208,80593.5,17,0
1645,둔촌역전통시장,200,13428152555,598080,127.134310,37.527818,372390.5,35,1
1646,길동복조리시장,133,5143279432,263420,127.143563,37.538925,348981.0,41,1
1647,명일전통시장,98,4766099294,259944,127.145337,37.549293,714887.5,69,2
